In [29]:
#scraping
import pymongo
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import re

In [30]:
!which chromedriver

/usr/local/bin/chromedriver


In [31]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [32]:
news_url='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(news_url)


In [33]:
html = browser.html
news_soup = bs(html,'html.parser')

In [34]:
#Nasa Mars News
news_title = news_soup.find_all("div", class_="content_title")[0].text
print (news_title)
news_paragraph = news_soup.body.find("div", class_="article_teaser_body").text
print(news_paragraph)

Mars Now
Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. 


In [35]:
#mars image
base_url = 'https://www.jpl.nasa.gov'
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(images_url)
html = browser.html
images_soup = bs(html, 'html.parser')

In [36]:
relative_image_path = images_soup.find_all('img')[3]["src"]
featured_images_url = base_url + relative_image_path
print(featured_images_url)
    

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24278-640x350.jpg


In [37]:
#Mars Facts
Facts_url = 'https://space-facts.com/mars/'
browser.visit(Facts_url)
time.sleep(2)

In [38]:
html = browser.html

table = pd.read_html(html)

facts_df = table[0]
facts_df.columns = ['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [39]:
facts_df.to_html('marsfacts.html', index=False)

In [40]:
#mars hemisphere
usgs_url='https://astrogeology.usgs.gov'

hem_images_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hem_images_url)

hem_html = browser.html

hem_soup = bs(hem_html, 'html.parser')


In [41]:
all_mars_hem = hem_soup.find('div', class_ = 'collapsible results')
mars_hem = all_mars_hem.find_all('div', class_='item')

hem_url = []

for i in mars_hem:
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text
    
    hem_link = hemisphere.a["href"]
    browser.visit(usgs_url + hem_link)
    
    image_html = browser.html
    image_soup=bs(image_html,'html.parser')
    
    image_link = image_soup.find('div',class_='downloads')
    image_url = image_link.find('li').a['href']
    
    
    image_dict={}
    image_dict['title']=title
    image_dict['img_url']=image_url
    
    hem_url.append(image_dict)

print(hem_url)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [46]:
mars_dict = {
    "news_title": news_title,
    "news_paragraph": news_paragraph,
    "featured_images_url":featured_images_url,
    "facts_df":str(table),
    "hemisphere_images":hem_url
    
}

In [48]:
mars_dict

{'news_title': 'Mars Now',
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 'featured_images_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24278-640x350.jpg',
 'facts_df': '[            Description                          Value\n0  Equatorial Diameter:                       6,792 km\n1       Polar Diameter:                       6,752 km\n2                 Mass:  6.39 × 10^23 kg (0.11 Earths)\n3                Moons:            2 (Phobos & Deimos)\n4       Orbit Distance:       227,943,824 km (1.38 AU)\n5         Orbit Period:           687 days (1.9 years)\n6  Surface Temperature:                   -87 to -5 °C\n7         First Record:              2nd millennium BC\n8          Recorded By:           Egyptian astronomers,   Mars - Earth Comparison             Mars            Earth\n0               Diameter:         6,779 km      